This notebook extract Image and Text Embeddings for further be used in cluserization methods

Embedding Extracted:
- BERT (Trained on Social Media)
- MobileNetV3 (Trained on ImageNet)
- MobileNetV3 + Bert Concat
- CLIP (Concat)
- CLIP (SUM)


# Extracting Bert Embeddings

In [3]:
# load data
import pandas as pd
import numpy as np
tweet_data = pd.read_pickle('../annotations/5k-data.pkl')

In [5]:
# preprocess
import preprocess
clean_text = [] 

for index, row in tweet_data.iterrows():
    clean_text.append(preprocess.pre_process(row['text'], keep_hashtag = True, keep_special_symbols = True))

tweet_data['processed_tweet'] = clean_text

In [ ]:
# Create Embeddings
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from sentence_transformers import SentenceTransformer, util

# Get pretrained model
pretrain_model_path = 'cardiffnlp/twitter-roberta-base'
model = SentenceTransformer(pretrain_model_path, cache_folder='.')

# Enccode text
tweets_emds = model.encode(tweet_data['processed_tweet'].tolist())

In [23]:
tweets_emds.shape

(5000, 768)

In [24]:
tweet_data['text_embeddings'] = tweets_emds.tolist()
tweet_data.to_pickle('tweet_data_text_emb.pkl')

# Image embeddings with MobileNetV3

In [12]:
from forensic_lib.forensicsEvidences.imgEv.CNN import  load_default_image_model_or_preprocess, get_image_embedding
from forensic_lib.utils.img_utils import load_image
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np

In [13]:
tweet_data

,tweet_id,work_time,orig_path,text,img,processed_tweet,text_embeddings
0,1437188092683751424,23.626190,social-data/media_12/pt/1437188092683751429/im...,Simone ja esta gravando um novo clipe para “En...,annotated-data/1437188092683751429.jpg,Simone ja esta gravando um novo clipe para “ E...,"[0.1138482466340065, 0.16404683887958527, 0.07..."
1,1435028621362614272,23.626190,social-data/media_07/pt/1435028621362614277/im...,"✅ Dominado Vila Real, Vila Pouca De Aguiar, Pe...",annotated-data/1435028621362614277.jpg,"Dominado Vila Real, Vila Pouca De Aguiar, Pens...","[0.00721816485747695, 0.14304794371128082, 0.0..."
2,1435346675674013696,23.626190,social-data/media_07/pt/1435346675674013701/im...,EVANGELINE LILLY EU TE AMO EU TE AMOOOO https...,annotated-data/1435346675674013701.jpg,EVANGELINE LILLY EU TE AMO EU TE AMOOOO,"[0.07994440197944641, 0.04783586412668228, 0.0..."
3,1437130100718383104,23.626190,social-data/media_12/pt/1437130100718383105/im...,"Aqui de boa na rede, enquanto o MBL passa verg...",annotated-data/1437130100718383105.jpg,"Aqui de boa na rede, enquanto o MBL passa verg...","[0.07642020285129547, 0.1480882316827774, 0.04..."
4,1435211167924957184,21.878664,social-data/media_07/en/1435211167924957190/im...,"@Khayizasoze I was sent this picture,sengiyasa...",annotated-data/1435211167924957190.jpg,"I was sent this picture,sengiyasaba","[0.14479757845401764, 0.04978952184319496, -0...."
...,...,...,...,...,...,...,...
4995,1436688481242255360,15.889647,social-data/media_8_11/pt/1436688481242255361/...,@Mano_Cebola1 ELE D NOVO SIIIIIM! E AINDA TEM ...,annotated-data/1436688481242255361.jpg,ELE D NOVO SIIIIIM! E AINDA TEM MAIS FIDDLES F...,"[0.07685352116823196, -0.006391509436070919, 0..."
4996,1436071900803915776,20.498567,social-data/media_8_11/pt/1436071900803915776/...,"Pois, é! Máscaras caindo… #BolsonaroArregou e ...",annotated-data/1436071900803915776.jpg,"Pois, é! Máscaras caindo… Bolsonaro Arregou e ...","[0.03707044944167137, 0.12567757070064545, 0.0..."
4997,1436067261094236160,20.498567,social-data/media_8_11/pt/1436067261094236169/...,Um móvel novo é sempre bem vindo para renovar ...,annotated-data/1436067261094236169.jpg,Um móvel novo é sempre bem vindo para renovar ...,"[0.06778176873922348, 0.14513982832431793, 0.0..."
4998,1435440766868987904,20.498567,social-data/media_8_11/pt/1435440766868987907/...,Descobri que zoar Bolsominions é bom pro engaj...,annotated-data/1435440766868987907.jpg,Descobri que zoar Bolsominions é bom pro engaj...,"[0.03715939074754715, 0.1534595936536789, 0.02..."


In [28]:
image_paths = [f'../annotations/{m}' for m in tweet_data['img'].values]
image_ids = [ i  for i in tweet_data.index]
model, m_transform = load_default_image_model_or_preprocess()
embeddings, embeddings_ids  = get_image_embedding(image_paths, image_ids, model, m_transform, normalize=True, gpu_id=0, use_gpu=True)

Model on GPU              


  0%|                                                                                                                                  | 0/79 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 78/79 [00:18<00:00,  4.47it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:18<00:00,  4.19it/s]

Model on CPU             


In [30]:
tweet_data['imgs_embeddings'] = embeddings

In [32]:
tweet_data.to_pickle('tweet_data_img_emb.pkl')

# Getting CLIP embeddings

Clip multilingual
@inproceedings{reimers-2019-sentence-bert,
    title = "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks",
    author = "Reimers, Nils and Gurevych, Iryna",
    booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing",
    month = "11",
    year = "2019",
    publisher = "Association for Computational Linguistics",
    url = "http://arxiv.org/abs/1908.10084",
}

## Image Clip Embeddings

In [44]:
import pandas as pd
import numpy as np

# read data
tweet_data = pd.read_pickle('tweet_data_img_emb.pkl')

In [89]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import clip
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from sentence_transformers import SentenceTransformer, util
from forensic_lib.forensicsEvidences.imgEv.CNN import  get_image_embedding, ImageDataset, normalize_vector
from forensic_lib.utils.img_utils import load_image
from tqdm import tqdm

In [86]:
# load model and image preprocessing
device = "cuda:0" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)


In [90]:
"""
Define image embedings extraction for clip
"""
def get_clip_image_embedding(
                        image_files,
                        image_ids,
                        clip_model,
                        transform,
                        normalize = False, 
                        **kwargs
):
    
    use_gpu = kwargs.get('use_gpu') if kwargs.get('use_gpu') else False
    
    if torch.cuda.is_available() and use_gpu: 
        dev = "cuda:"+ str(kwargs.get('gpu_id')) if kwargs.get('gpu_id') else 'cuda:0'
    else: 
        dev = "cpu" 
    
    # No need to create overhead using batch
    if len(image_files) == 1:
        if check_ext(image_files[0]):
            img = transform(load_image(image_files[0])).unsqueeze(0) 
            
            with torch.no_grad():
                embedded_vectors = clip_model.encode_image(img).squeeze().numpy()
                
            if normalize:
                embedded_vectors = normalize_vector(embedded_vectors)
            return [list(embedded_vectors)], image_ids
    
    # Batch of images
    else:
        embedded_vectors = []
        ids = []
        img_dataset = ImageDataset(image_files, image_ids, transform)
        
        if use_gpu:    
            print("\rLoading Model to Device\r",end="")
            clip_model = clip_model.to(dev)
            print("Model on GPU              ",flush=True)

        batch_size = kwargs.get('batch_size') if kwargs.get('batch_size') else 64
        num_workers = kwargs.get('num_workers') if kwargs.get('num_workers') else 4
        img_dataloader = DataLoader(img_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
        
        for imgs, ids_batch in tqdm(img_dataloader, total=np.ceil(len(img_dataset) / batch_size).astype(int)):
            imgs = imgs.to(dev)
            
            with torch.no_grad():
                output = clip_model.encode_image(imgs)
                output = output.squeeze().cpu().numpy()
                
            if normalize:
                output = normalize_vector(output)
                
            embedded_vectors += output.tolist()
            ids += [int(_id) for _id in ids_batch]
            
        if use_gpu:    
            print("\rRemove Model from Device\r",end="")
            clip_model = clip_model.to('cpu')
            print("Model on CPU             ",flush=True)
            
        return embedded_vectors, ids   



In [91]:
image_paths = [f'../annotations/{m}' for m in tweet_data['img'].values]
image_ids = [ i  for i in tweet_data.index]

img_clip_embeddings, img_clip_embeddings_ids  = get_clip_image_embedding(image_paths, image_ids, clip_model, preprocess, normalize=True, gpu_id=0, use_gpu=True)

Model on GPU              


  0%|                                                                                                                                  | 0/79 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 78/79 [00:23<00:00,  3.37it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:23<00:00,  3.39it/s]


Model on CPU             


In [92]:
tweet_data['clip_imgs_embeddings'] = img_clip_embeddings

In [73]:
tweet_data.to_pickle('tweet_data_clip_img_emb.pkl')

## Text Clip embeddings

In [77]:
text_model = SentenceTransformer('sentence-transformers/clip-ViT-B-32-multilingual-v1',cache_folder='.', device='cuda:0')

In [81]:
# Enccode text
tweets_emds = text_model.encode(tweet_data['processed_tweet'].tolist())

tweet_data['clip_text_embeddings'] = tweets_emds.tolist()

In [83]:
tweet_data['clip_text_embeddings'] = tweets_emds.tolist()

In [97]:
tweet_data.to_pickle('tweet_data_all_emb.pkl')

In [99]:
tweet_data

,tweet_id,work_time,orig_path,text,img,processed_tweet,text_embeddings,imgs_embeddings,clip_imgs_embeddings,clip_text_embeddings
0,1437188092683751424,23.626190,social-data/media_12/pt/1437188092683751429/im...,Simone ja esta gravando um novo clipe para “En...,annotated-data/1437188092683751429.jpg,Simone ja esta gravando um novo clipe para “ E...,"[0.1138482466340065, 0.16404683887958527, 0.07...","[0.0017162024741992354, 0.017185570672154427, ...","[0.001071929931640625, 0.002445220947265625, -...","[0.1509532630443573, -0.14887511730194092, -0...."
1,1435028621362614272,23.626190,social-data/media_07/pt/1435028621362614277/im...,"✅ Dominado Vila Real, Vila Pouca De Aguiar, Pe...",annotated-data/1435028621362614277.jpg,"Dominado Vila Real, Vila Pouca De Aguiar, Pens...","[0.00721816485747695, 0.14304794371128082, 0.0...","[0.02427668683230877, -0.010925845243036747, 0...","[-0.01496124267578125, -0.0121307373046875, 0....","[-0.1372763216495514, 0.11387073993682861, 0.0..."
2,1435346675674013696,23.626190,social-data/media_07/pt/1435346675674013701/im...,EVANGELINE LILLY EU TE AMO EU TE AMOOOO https...,annotated-data/1435346675674013701.jpg,EVANGELINE LILLY EU TE AMO EU TE AMOOOO,"[0.07994440197944641, 0.04783586412668228, 0.0...","[0.0498591773211956, 0.0070304046384990215, -0...","[0.0223388671875, -0.0005679130554199219, -0.0...","[0.026219116523861885, 0.10483500361442566, -0..."
3,1437130100718383104,23.626190,social-data/media_12/pt/1437130100718383105/im...,"Aqui de boa na rede, enquanto o MBL passa verg...",annotated-data/1437130100718383105.jpg,"Aqui de boa na rede, enquanto o MBL passa verg...","[0.07642020285129547, 0.1480882316827774, 0.04...","[0.010791047476232052, 0.033861253410577774, 0...","[0.030792236328125, -0.0093994140625, 0.004520...","[0.2534538209438324, 0.15335972607135773, -0.1..."
4,1435211167924957184,21.878664,social-data/media_07/en/1435211167924957190/im...,"@Khayizasoze I was sent this picture,sengiyasa...",annotated-data/1435211167924957190.jpg,"I was sent this picture,sengiyasaba","[0.14479757845401764, 0.04978952184319496, -0....","[0.03496298938989639, 0.011268963105976582, -0...","[-0.0131988525390625, 0.007293701171875, -0.01...","[0.09891669452190399, 0.15106208622455597, 0.1..."
...,...,...,...,...,...,...,...,...,...,...
4995,1436688481242255360,15.889647,social-data/media_8_11/pt/1436688481242255361/...,@Mano_Cebola1 ELE D NOVO SIIIIIM! E AINDA TEM ...,annotated-data/1436688481242255361.jpg,ELE D NOVO SIIIIIM! E AINDA TEM MAIS FIDDLES F...,"[0.07685352116823196, -0.006391509436070919, 0...","[0.01883683167397976, -0.014022131450474262, 0...","[-0.03485107421875, 0.0283660888671875, 0.0211...","[-0.01074286364018917, 0.007608398795127869, 0..."
4996,1436071900803915776,20.498567,social-data/media_8_11/pt/1436071900803915776/...,"Pois, é! Máscaras caindo… #BolsonaroArregou e ...",annotated-data/1436071900803915776.jpg,"Pois, é! Máscaras caindo… Bolsonaro Arregou e ...","[0.03707044944167137, 0.12567757070064545, 0.0...","[0.033475033938884735, -0.005199815612286329, ...","[-0.004947662353515625, 0.042572021484375, -0....","[0.04612525552511215, 0.11057652533054352, -0...."
4997,1436067261094236160,20.498567,social-data/media_8_11/pt/1436067261094236169/...,Um móvel novo é sempre bem vindo para renovar ...,annotated-data/1436067261094236169.jpg,Um móvel novo é sempre bem vindo para renovar ...,"[0.06778176873922348, 0.14513982832431793, 0.0...","[0.03648684173822403, 0.038903187960386276, 0....","[-0.0277557373046875, 0.043914794921875, 0.007...","[0.029536902904510498, 0.23566432297229767, -0..."
4998,1435440766868987904,20.498567,social-data/media_8_11/pt/1435440766868987907/...,Descobri que zoar Bolsominions é bom pro engaj...,annotated-data/1435440766868987907.jpg,Descobri que zoar Bolsominions é bom pro engaj...,"[0.03715939074754715, 0.1534595936536789, 0.02...","[0.02073543332517147, -0.009916947223246098, 0...","[0.0243682861328125, 0.08709716796875, 0.01355...","[0.12946784496

# Create concat and sum of embeddings

In [109]:
tweet_data = tweet_data.rename(columns={'imgs_embeddings':'MobileNet_embeddings', 'text_embeddings': 'Roberta_embeddings'})

In [118]:
MobileNet_Roberta_embeddings = []
clip_imgs_cat_text_embeddings = []
clip_imgs_sum_text_embeddings = []
for index, row in tweet_data.iterrows():
    robert_emb = row['Roberta_embeddings']
    mnet_emb = row['MobileNet_embeddings']
    MobileNet_Roberta_embeddings.append(mnet_emb + robert_emb)
    
    clip_img_emb = row['clip_imgs_embeddings']
    clip_txt_emb = row['clip_text_embeddings']
    clip_imgs_cat_text_embeddings.append(clip_img_emb + clip_txt_emb)
    clip_imgs_sum_text_embeddings.append((np.array(clip_img_emb) + np.array(clip_txt_emb)).tolist())


In [120]:

tweet_data['MobileNet_Roberta_embeddings'] = MobileNet_Roberta_embeddings
tweet_data['clip_imgs_cat_text_embeddings'] = clip_imgs_cat_text_embeddings
tweet_data['clip_imgs_sum_text_embeddings'] = clip_imgs_sum_text_embeddings

In [121]:
tweet_data.to_pickle('tweet_data_all_emb.pkl')